# Data Preprocessing
## Overview
1. Read recipe data that was scraped from some recipe website
2. Combine different scraped datasets into a single file to be read in by the database
3. Clean ingredient data by extracting out only the ingredients to be passed to TF-IDF, etc.

In [1]:
import pandas as pd
import numpy as np

import os
import re
import glob

## 1. Aggregate scraped data

### Load in data

Point to the directory where the recipe CSVs are stored

In [2]:
DATADIR = '../data/'
recipe_filenames = glob.glob(DATADIR + 'recipe_info_dataset*')

In [3]:
recipe_filenames

['../data/recipe_info_dataset6.csv',
 '../data/recipe_info_dataset5.csv',
 '../data/recipe_info_dataset4.csv',
 '../data/recipe_info_dataset3.csv',
 '../data/recipe_info_dataset2.csv']

In [4]:
for idx, filename in enumerate(recipe_filenames):
    
    temp_df = pd.read_csv(filename, header=0)
    
    if idx == 0:
        df = temp_df
        
    else:
        df = df.append(temp_df, ignore_index=True)

In [5]:
df.shape

(17777, 8)

### Sample the data
- We don't need this much data to demonstrate, so let's take a sample

In [6]:
df = df.sample(n=1000, random_state=41234)

### Convert calories and review counts into numbers

In [7]:
df['calorie_count'] = pd.to_numeric( df['calorie_count'].str.replace("cals",""), errors='coerce')

In [8]:
df['review_count'] = pd.to_numeric( df['review_count'].str.replace("reviews",""), errors='coerce')

### Remove recipes with no calorie count

In [9]:
df.dropna(subset=['calorie_count'], inplace=True)

Take a look at what came out:

In [10]:
df.head()

,recipe_id,recipe_name,recipe_url,ingredients,cook_time,calorie_count,review_count,overall_rating
14932,257594,sloppy-joes-from-ball-park-buns,https://www.allrecipes.com/recipe/257594/slopp...,"['4 Ball Park® Tailgaters Gourmet Buns', '1 po...",50 m,578.0,7,4.285714
15237,257968,chilaquiles,https://www.allrecipes.com/recipe/257968/chila...,"['2 tomatoes, chopped', '6 dried chile de arbo...",33 m,329.0,3,3.750000
10018,235479,perfectly-roasted-vegetable,https://www.allrecipes.com/recipe/235479/perfe...,"['2 cups Brussels sprouts, trimmed', '1 cup la...",1 h,105.0,17,4.555555
14611,257248,jeffreys-eggs-in-a-basket,https://www.allrecipes.com/recipe/257248/jeffr...,"['6 slices bread', '1/4 cup butter, at room te...",13 m,232.0,1,5.000000
5157,230310,coyote-dip,https://www.allrecipes.com/recipe/230310/coyot...,['1 (8 ounce) package reduced-fat cream cheese...,2 h 15 m,164.0,9,4.727273


### Save this merged table to be read into the database

In [11]:
df.to_csv('../FoodFlix/static/data/recipes_all_data.csv')

## 2. Prepare data to be used in recommendation system
- Now we only need the ingredients and the recipe id to use as a key

In [12]:
df = df[['recipe_id', 'ingredients', 'recipe_name', 'calorie_count', 'cook_time', 'recipe_url', 'review_count', 'overall_rating']]
df['ingredients_raw'] = df['ingredients'] #We want the 'uncleaned' ingredients for display
df.set_index('recipe_id', inplace=True)

### Remove all non-alphabetical characters

In [13]:
def clean_ingredients(ingredient):
    return re.sub("[^a-zA-Z]", " ", ingredient).strip().lower()

In [14]:
df['ingredients'] = df['ingredients'].apply(clean_ingredients)

### Remove stop words

In [15]:
with open('../data/stopwords.txt', 'r') as infile:
    stop_words = infile.read().split('\n')

In [16]:
def remove_stop_words(ingredients):
    clean = [k for k in ingredients.split() if k not in stop_words]
    clean = ' '.join(clean).strip()
    return clean

In [17]:
df['ingredients'] = df['ingredients'].apply(remove_stop_words)

### Check if the recipe names are unique:

In [18]:
len(df)

994

In [19]:
len(df['recipe_name'].unique())

994

Let's drop the duplicates (if any):

In [20]:
df = df[~df['recipe_name'].duplicated()]

In [21]:
df.shape

(994, 8)

### Remove empty ingredient lists

In [22]:
df = df[df['ingredients'].apply(lambda x: bool(x))]

In [23]:
df.shape

(994, 8)

In [24]:
df.head()

,ingredients,recipe_name,calorie_count,cook_time,recipe_url,review_count,overall_rating,ingredients_raw
recipe_id,,,,,,,,
257594,park tailgaters gourmet buns ground beef yello...,sloppy-joes-from-ball-park-buns,578.0,50 m,https://www.allrecipes.com/recipe/257594/slopp...,7,4.285714,"['4 Ball Park® Tailgaters Gourmet Buns', '1 po..."
257968,tomatoes dried chile de arbol peppers onion ga...,chilaquiles,329.0,33 m,https://www.allrecipes.com/recipe/257968/chila...,3,3.750000,"['2 tomatoes, chopped', '6 dried chile de arbo..."
235479,brussels sprouts yukon gold potato chunks rain...,perfectly-roasted-vegetable,105.0,1 h,https://www.allrecipes.com/recipe/235479/perfe...,17,4.555555,"['2 cups Brussels sprouts, trimmed', '1 cup la..."
257248,bread butter eggs,jeffreys-eggs-in-a-basket,232.0,13 m,https://www.allrecipes.com/recipe/257248/jeffr...,1,5.000000,"['6 slices bread', '1/4 cup butter, at room te..."
230310,reduced fat cream cheese reduced fat mayonnais...,coyote-dip,164.0,2 h 15 m,https://www.allrecipes.com/recipe/230310/coyot...,9,4.727273,['1 (8 ounce) package reduced-fat cream cheese...


### Save the cleaned ingredients to file

In [25]:
filename = '../FoodFlix/static/data/clean_ingredients.csv'
df.to_csv(filename)
#Check that file exists
if not os.path.isfile(filename):
    print('Failed to create ',filename)
    raise